In [6]:
#pip install tensorflow

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold

# 1. Importation de la data

In [2]:
#Récup de tous les matchs dans un dataframe

In [3]:
versus=pd.read_csv("matchcoupedumonde.csv",sep=';')

In [4]:
versus

,date,home_team,home_score,away_score,away_team,tournament,winner,phase_final,home_team_pourcentage_victoire,home_team_but_mis,home_team_but_enc,away_team_pourcentage_victoire,away_team_but_mis,away_team_but_enc,Rang_home_team,Rang_away_team,Host
0,1930,Spain,1,0,Czechoslovakia,0,2,0,85.71,3.1,1.1,37.50,2.2,1.7,NaN,NaN,1
1,1930,Portugal,1,0,Czechoslovakia,0,2,0,28.57,0.7,2.3,37.50,2.2,1.7,NaN,NaN,1
2,1930,Greece,2,1,Yugoslavia,0,2,0,50.00,2.5,2.8,41.67,2.4,2.1,NaN,NaN,1
3,1930,Northern Ireland,7,0,Wales,0,2,0,16.67,2.3,3.3,0.00,0.8,4.0,NaN,NaN,1
4,1930,Italy,4,2,Switzerland,0,2,0,55.56,2.8,1.6,7.69,1.7,3.7,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29648,2021,Tunisia,1,0,Egypt,0,2,0,68.42,1.7,0.7,60.00,1.8,0.5,26.0,46.0,1
29649,2021,Autre pays,1,2,Algeria,0,0,0,20.00,0.8,1.8,70.00,3.0,0.8,99.0,33.0,1
29650,2021,Autre pays,0,0,Egypt,0,1,0,20.00,0.8,1.8,60.00,1.8,0.5,99.0,46.0,1
29651,2021,Tunisia,0,2,Algeria,0,0,0,68.42,1.7,0.7,70.00,3.0,0.8,26.0,33.0,1


In [5]:
#Sélection des matchs de coupe du monde seulement

In [6]:
versus=versus[versus['tournament']==1]

In [7]:
#Affichage des matchs de coupe du monde

In [8]:
versus

,date,home_team,home_score,away_score,away_team,tournament,winner,phase_final,home_team_pourcentage_victoire,home_team_but_mis,home_team_but_enc,away_team_pourcentage_victoire,away_team_but_mis,away_team_but_enc,Rang_home_team,Rang_away_team,Host
43,1930,Belgium,0,3,United States,1,0,0,38.46,2.2,2.2,0.00,3.0,4.0,NaN,NaN,0
44,1930,France,4,1,Mexico,1,2,0,21.43,1.5,2.9,NaN,NaN,NaN,NaN,NaN,0
45,1930,Brazil,1,2,Yugoslavia,1,0,0,100.00,3.7,2.0,41.67,2.4,2.1,NaN,NaN,0
46,1930,Peru,1,3,Romania,1,0,0,0.00,0.3,4.0,57.14,3.1,2.0,NaN,NaN,0
47,1930,Argentina,1,0,France,1,2,0,55.56,1.9,0.7,21.43,1.5,2.9,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27719,2018,Russia,3,4,Croatia,1,0,1,28.57,1.4,1.5,40.91,1.0,1.2,70.0,20.0,1
27720,2018,France,1,0,Belgium,1,2,1,59.09,1.9,1.0,70.00,2.8,1.0,7.0,3.0,0
27721,2018,Croatia,2,1,England,1,2,1,40.91,1.0,1.2,60.00,1.5,0.7,20.0,12.0,0
27722,2018,Belgium,2,0,England,1,2,1,70.00,2.8,1.0,60.00,1.5,0.7,3.0,12.0,0


In [9]:
#Réindexation du dataframe

In [10]:
versus=versus.reset_index(inplace=False)

In [11]:
#versus

In [12]:
#Calcul des stats de victoire a domicile exterieur et match nul

In [13]:
victoire_home = 0
victoire_away = 0
nul = 0
for i in range (len(versus)) :
    if versus.loc[i]["home_score"] == versus.loc[i]["away_score"]:
        nul +=1
    elif versus.loc[i]["home_score"] > versus.loc[i]["away_score"] :
        victoire_home +=1
    elif versus.loc[i]["home_score"] < versus.loc[i]["away_score"] :
        victoire_away +=1
    
poucentagevict_home = (victoire_home/ len(versus))*100
pourcentagevict_away = (victoire_away/ len(versus))*100
pourcentage_nul = (nul/len(versus))*100

In [14]:
print('Pourcentage de victoire a domicile :',poucentagevict_home)

Pourcentage de victoire a domicile : 47.333333333333336


In [15]:
print("Pourcentage de victoire a l'extérieur :",pourcentagevict_away)

Pourcentage de victoire a l'extérieur : 33.0


In [16]:
print('Pourcentage de match nul',pourcentage_nul)

Pourcentage de match nul 19.666666666666664


# 2.Traitement de la data

In [17]:
versus=versus.drop('home_score',axis=1)

In [18]:
versus=versus.drop('away_score',axis=1)

In [19]:
versus=versus.drop('home_team',axis=1)

In [20]:
versus=versus.drop('away_team',axis=1)

In [21]:
versus

,index,date,tournament,winner,phase_final,home_team_pourcentage_victoire,home_team_but_mis,home_team_but_enc,away_team_pourcentage_victoire,away_team_but_mis,away_team_but_enc,Rang_home_team,Rang_away_team,Host
0,43,1930,1,0,0,38.46,2.2,2.2,0.00,3.0,4.0,NaN,NaN,0
1,44,1930,1,2,0,21.43,1.5,2.9,NaN,NaN,NaN,NaN,NaN,0
2,45,1930,1,0,0,100.00,3.7,2.0,41.67,2.4,2.1,NaN,NaN,0
3,46,1930,1,0,0,0.00,0.3,4.0,57.14,3.1,2.0,NaN,NaN,0
4,47,1930,1,2,0,55.56,1.9,0.7,21.43,1.5,2.9,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,27719,2018,1,0,1,28.57,1.4,1.5,40.91,1.0,1.2,70.0,20.0,1
896,27720,2018,1,2,1,59.09,1.9,1.0,70.00,2.8,1.0,7.0,3.0,0
897,27721,2018,1,2,1,40.91,1.0,1.2,60.00,1.5,0.7,20.0,12.0,0
898,27722,2018,1,2,1,70.00,2.8,1.0,60.00,1.5,0.7,3.0,12.0,0


In [22]:
versus.dropna(axis=0,inplace=True)

In [23]:
versus['Rang_home_team']=(1.)/(versus['Rang_home_team'])

In [24]:
versus['Rang_away_team']=(1.)/(versus['Rang_away_team'])

In [25]:
versus

,index,date,tournament,winner,phase_final,home_team_pourcentage_victoire,home_team_but_mis,home_team_but_enc,away_team_pourcentage_victoire,away_team_but_mis,away_team_but_enc,Rang_home_team,Rang_away_team,Host
464,13493,1994,1,2,0,71.43,2.2,0.9,27.03,1.2,1.2,1.000000,0.023256,0
465,13494,1994,1,1,0,73.68,2.2,0.7,45.71,1.7,0.8,0.200000,0.027027,0
466,13495,1994,1,0,0,53.33,1.5,0.5,52.17,1.7,1.1,0.058824,0.142857,0
467,13496,1994,1,0,0,70.59,1.9,0.6,66.67,1.8,0.8,0.250000,0.071429,0
468,13497,1994,1,1,0,29.09,1.3,1.2,65.00,1.7,0.6,0.043478,0.083333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,27719,2018,1,0,1,28.57,1.4,1.5,40.91,1.0,1.2,0.014286,0.050000,1
896,27720,2018,1,2,1,59.09,1.9,1.0,70.00,2.8,1.0,0.142857,0.333333,0
897,27721,2018,1,2,1,40.91,1.0,1.2,60.00,1.5,0.7,0.050000,0.083333,0
898,27722,2018,1,2,1,70.00,2.8,1.0,60.00,1.5,0.7,0.333333,0.083333,0


# 3.Convertion du dataframe en tableau

In [26]:
#Met y sous forme d'un tableau
y=versus[['winner']].values


In [27]:
#je garde que trois variables pour le model Rang fifa équipe 1 et rang fifa équipe 2 et une variable si le match se joue à domicile

In [28]:
X=versus[['Rang_home_team','Rang_away_team','Host']].values
X.shape

(426, 3)

# 4.Cross validation

In [29]:
for train_index,test_index in KFold(3,shuffle=True, random_state=0).split(X):
    X_train,X_test=X[train_index],X[test_index]
    y_train,y_test=y[train_index],y[test_index]

# 5.Création d'un réseau de Neurone Artificiel

In [30]:
model = Sequential()

model.add(Dense(32, input_dim=3))
model.add(Activation('relu'))


model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam',metrics=['sparse_categorical_accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                128       
                                                                 
 activation (Activation)     (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
 activation_1 (Activation)   (None, 3)                 0         
                                                                 
Total params: 227
Trainable params: 227
Non-trainable params: 0
_________________________________________________________________


# 6.entraînement du model

In [31]:
model.fit(X_train,y_train,epochs=100,batch_size=1,verbose=1)

Epoch 1/100
284/284 [==============================] - 1s 693us/step - loss: 1.0587 - sparse_categorical_accuracy: 0.4296
Epoch 2/100
284/284 [==============================] - 0s 703us/step - loss: 1.0005 - sparse_categorical_accuracy: 0.5070
Epoch 3/100
284/284 [==============================] - 0s 678us/step - loss: 0.9887 - sparse_categorical_accuracy: 0.5070
Epoch 4/100
284/284 [==============================] - 0s 703us/step - loss: 0.9834 - sparse_categorical_accuracy: 0.5070
Epoch 5/100
284/284 [==============================] - 0s 689us/step - loss: 0.9775 - sparse_categorical_accuracy: 0.5070
Epoch 6/100
284/284 [==============================] - 0s 682us/step - loss: 0.9738 - sparse_categorical_accuracy: 0.5070
Epoch 7/100
284/284 [==============================] - 0s 671us/step - loss: 0.9702 - sparse_categorical_accuracy: 0.5106
Epoch 8/100
284/284 [==============================] - 0s 678us/step - loss: 0.9665 - sparse_categorical_accuracy: 0.5211
Epoch 9/100
284/284 [===

284/284 [==============================] - 0s 717us/step - loss: 0.9145 - sparse_categorical_accuracy: 0.5669
Epoch 68/100
284/284 [==============================] - 0s 707us/step - loss: 0.9124 - sparse_categorical_accuracy: 0.5669
Epoch 69/100
284/284 [==============================] - 0s 760us/step - loss: 0.9127 - sparse_categorical_accuracy: 0.5669
Epoch 70/100
284/284 [==============================] - 0s 686us/step - loss: 0.9113 - sparse_categorical_accuracy: 0.5810
Epoch 71/100
284/284 [==============================] - 0s 693us/step - loss: 0.9118 - sparse_categorical_accuracy: 0.5775
Epoch 72/100
284/284 [==============================] - 0s 689us/step - loss: 0.9109 - sparse_categorical_accuracy: 0.5739
Epoch 73/100
284/284 [==============================] - 0s 693us/step - loss: 0.9104 - sparse_categorical_accuracy: 0.5739
Epoch 74/100
284/284 [==============================] - 0s 700us/step - loss: 0.9104 - sparse_categorical_accuracy: 0.5775
Epoch 75/100
284/284 [=======

# 7.Evaluation du model sur la data de test

In [32]:
test_loss, test_accuracy= model.evaluate(X_test,y_test)

5/5 [==============================] - 0s 1000us/step - loss: 1.0884 - sparse_categorical_accuracy: 0.5211


# Sauvegarde du Model

In [79]:
model.save("fifa2022.h5")

# Chargement du model

In [ ]:
#Permet de travaillé avec un model déjà entraîné 

In [120]:
model=tf.keras.models.load_model('fifa2022.h5')

# 8.Prédiction / Chargement de la data à prédire

In [33]:
#Chargement des matchs des coupe du monde de 2010 à 2018
#Permet de tester notre model sur les dernieres coupes du monde
#Ajout des stats des cotes des matchs
#Permet de voir si le model peut raporter de l'argent

In [34]:
match_precedent_coupe=pd.read_excel("pronoanciennecoupe.xlsx")

In [35]:
poule=match_precedent_coupe

In [36]:
poule=poule.drop('winner',axis=1)
poule_Pays=match_precedent_coupe


In [37]:
poule=poule.drop('home_team',axis=1)

In [38]:
poule=poule.drop('away_team',axis=1)

In [39]:
poule['Rang_home_team']=(1.)/(poule['Rang_home_team'])

In [40]:
poule['Rang_away_team']=(1.)/(poule['Rang_away_team'])

In [41]:
X_pred=poule[['Rang_home_team','Rang_away_team','Host']].values


# Affichage de la prédiction des matchs de poule

In [42]:
#Lance la prédiction du match
#Ajoute la prédiction à la fin du dataframe (colonne pred)
#Affiche si la prédiction est bonne (colonne trouve)
#Affiche le gain ou la perte si pari sportif(colonne prono)

In [43]:
prediction=model.predict(X_pred)
i=0

for pred in prediction:
    print('Pour le match entre',poule_Pays['home_team'][i],'et',poule_Pays['away_team'][i])
    if (np.argmax(pred))==2 :

        print('Victoire pour',poule_Pays['home_team'][i],'\n')
        match_precedent_coupe['pred'][i]=2
        if match_precedent_coupe['winner'][i]==match_precedent_coupe['pred'][i] :
            match_precedent_coupe['trouve'][i]=1
        else :
            match_precedent_coupe['trouve'][i]=0
        
        if match_precedent_coupe['trouve'][i]==0. :
            match_precedent_coupe['prono'][i]=-1
        else :
            if match_precedent_coupe['pred'][i]==2. :
                match_precedent_coupe['prono'][i]=(match_precedent_coupe['PronovicPays1'][i])-1
            elif match_precedent_coupe['pred'][i]==1. :
                match_precedent_coupe['prono'][i]=(match_precedent_coupe['Pronovicnul'][i])-1
            elif match_precedent_coupe['pred'][i]==0. :
                match_precedent_coupe['prono'][i]=(match_precedent_coupe['PronovicPays2'][i])-1
                
                
                


    elif (np.argmax(pred))==0:
        print('Victoire pour',poule_Pays['away_team'][i],'\n')
        match_precedent_coupe['pred'][i]=0
        if match_precedent_coupe['winner'][i]==match_precedent_coupe['pred'][i] :
            match_precedent_coupe['trouve'][i]=1
        else :
            match_precedent_coupe['trouve'][i]=0
        if match_precedent_coupe['trouve'][i]==0. :
            match_precedent_coupe['prono'][i]=-1
        else :
            if match_precedent_coupe['pred'][i]==2. :
                match_precedent_coupe['prono'][i]=(match_precedent_coupe['PronovicPays1'][i])-1
            elif match_precedent_coupe['pred'][i]==1. :
                match_precedent_coupe['prono'][i]=(match_precedent_coupe['Pronovicnul'][i])-1
            elif match_precedent_coupe['pred'][i]==0. :
                match_precedent_coupe['prono'][i]=(match_precedent_coupe['PronovicPays2'][i])-1

    else :
        print('Match null\n')
        match_precedent_coupe['pred'][i]=1
        if match_precedent_coupe['winner'][i]==match_precedent_coupe['pred'][i] :
            match_precedent_coupe['trouve'][i]=1
        else :
            match_precedent_coupe['trouve'][i]=0
        if match_precedent_coupe['trouve'][i]==0. :
            match_precedent_coupe['prono'][i]=-1
        else :
            if match_precedent_coupe['pred'][i]==2. :
                match_precedent_coupe['prono'][i]=(match_precedent_coupe['PronovicPays1'][i])-1
            elif match_precedent_coupe['pred'][i]==1. :
                match_precedent_coupe['prono'][i]=(match_precedent_coupe['Pronovicnul'][i])-1
            elif match_precedent_coupe['pred'][i]==0. :
                match_precedent_coupe['prono'][i]=(match_precedent_coupe['PronovicPays2'][i])-1


    i=i+1


6/6 [==============================] - 0s 1ms/step
Pour le match entre South Africa et Mexico
Victoire pour South Africa 

Pour le match entre Uruguay et France
Victoire pour France 

Pour le match entre Argentina et Nigeria
Victoire pour Argentina 

Pour le match entre England et United States
Victoire pour England 

Pour le match entre South Korea et Greece
Victoire pour Greece 

Pour le match entre Algeria et Slovenia
Victoire pour Algeria 

Pour le match entre Germany et Australia
Victoire pour Germany 

Pour le match entre Serbia et Ghana
Victoire pour Serbia 

Pour le match entre Italy et Paraguay
Victoire pour Italy 

Pour le match entre Japan et Cameroon
Victoire pour Cameroon 

Pour le match entre Netherlands et Denmark
Victoire pour Netherlands 

Pour le match entre Brazil et North Korea
Victoire pour Brazil 

Pour le match entre Ivory Coast et Portugal
Victoire pour Portugal 

Pour le match entre New Zealand et Slovakia
Victoire pour Slovakia 

Pour le match entre Honduras e

C:\Users\chris\AppData\Local\Temp\ipykernel_14032\1938671058.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_precedent_coupe['pred'][i]=2
C:\Users\chris\AppData\Local\Temp\ipykernel_14032\1938671058.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_precedent_coupe['trouve'][i]=0
C:\Users\chris\AppData\Local\Temp\ipykernel_14032\1938671058.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_precedent_coupe['pro

 Portugal et Morocco
Victoire pour Portugal 

Pour le match entre Uruguay et Saudi Arabia
Victoire pour Uruguay 

Pour le match entre Iran et Spain
Victoire pour Spain 

Pour le match entre Denmark et Australia
Victoire pour Denmark 

Pour le match entre France et Peru
Victoire pour France 

Pour le match entre Argentina et Croatia
Victoire pour Argentina 

Pour le match entre Brazil et Costa Rica
Victoire pour Brazil 

Pour le match entre Nigeria et Iceland
Victoire pour Iceland 

Pour le match entre Serbia et Switzerland
Victoire pour Switzerland 

Pour le match entre Belgium et Tunisia
Victoire pour Belgium 

Pour le match entre South Korea et Mexico
Victoire pour Mexico 

Pour le match entre Germany et Sweden
Victoire pour Germany 

Pour le match entre England et Panama
Victoire pour England 

Pour le match entre Japan et Senegal
Victoire pour Senegal 

Pour le match entre Poland et Colombia
Victoire pour Poland 

Pour le match entre Russia et Uruguay
Victoire pour Russia 

Pour le

In [44]:
#Affichage des pourcentage dans chaque catégorie avec le num du match

In [45]:
for i in range (30,60):
    print(prediction[i],i)

[0.10783312 0.12437596 0.7677909 ] 30
[0.14728342 0.0730297  0.7796869 ] 31
[0.4673498  0.17724308 0.3554071 ] 32
[0.36308482 0.22341914 0.41349602] 33
[0.31254047 0.25260484 0.43485466] 34
[0.1593689  0.23093918 0.60969186] 35
[0.38626048 0.2214926  0.392247  ] 36
[0.5975995  0.15411794 0.24828264] 37
[0.50788003 0.18402797 0.30809194] 38
[0.29366392 0.23869795 0.46763813] 39
[0.69012594 0.1089828  0.20089124] 40
[0.34714094 0.25506943 0.3977896 ] 41
[0.3235958  0.26109198 0.41531226] 42
[0.6560314  0.13150175 0.21246684] 43
[0.80070627 0.01778822 0.1815055 ] 44
[0.40623483 0.24411973 0.34964535] 45
[8.5459274e-01 4.9844361e-04 1.4490880e-01] 46
[0.3308128  0.25030836 0.41887882] 47
[0.28871796 0.24875553 0.4625265 ] 48
[0.29035524 0.23998164 0.4696631 ] 49
[0.2196207  0.20509619 0.5752831 ] 50
[0.2703174  0.17663024 0.55305237] 51
[0.28441098 0.01270084 0.7028882 ] 52
[0.10755044 0.15773988 0.7347097 ] 53
[0.33957037 0.25459886 0.40583086] 54
[0.4833065  0.04185001 0.47484353] 55
[7.

In [46]:
match_precedent_coupe[30:60]

,date,home_team,home_score,away_score,away_team,tournament,winner,phase_final,home_team_pourcentage_victoire,home_team_but_mis,...,away_team_but_enc,Rang_home_team,Rang_away_team,Host,PronovicPays1,Pronovicnul,PronovicPays2,pred,trouve,prono
30,2010,Portugal,7,0,North Korea,1,2,0,71.43,2.0,...,0.9,3,110,0,1.30,4,9.00,2.0,1.0,0.30
31,2010,Spain,2,0,Honduras,1,2,0,84.62,2.7,...,1.1,2,38,0,1.05,8.25,17.50,2.0,1.0,0.05
32,2010,Greece,0,2,Argentina,1,0,0,42.11,1.2,...,1.2,13,7,0,6.25,3.9,1.40,0.0,1.0,0.40
33,2010,Mexico,0,1,Uruguay,1,0,0,56.76,1.9,...,1.1,17,16,0,3.30,1.83,3.50,2.0,0.0,-1.00
34,2010,Nigeria,2,2,South Korea,1,1,0,45.00,1.3,...,0.7,21,47,0,2.75,3.1,2.25,2.0,0.0,-1.00
35,2010,South Africa,2,1,France,1,2,0,40.00,1.0,...,0.7,83,9,1,1.91,3.4,3.25,2.0,1.0,0.91
36,2010,Australia,2,1,Serbia,1,2,0,50.00,1.0,...,0.8,20,15,0,4.09,3.3,1.72,2.0,1.0,3.09
37,2010,Ghana,0,1,Germany,1,0,0,39.13,1.2,...,0.6,32,6,0,7.50,4,1.34,0.0,1.0,0.34
38,2010,Slovenia,0,1,England,1,0,0,55.56,1.8,...,0.8,25,8,0,8.00,3.9,1.34,0.0,1.0,0.34
39,2010,United States,1,0,Algeria,1,2,0,50.00,1.7,...,1.2,14,30,0,1.72,3.4,3.90,2.0,1.0,0.72


# Génération du fihier avec les pronos

In [47]:
match_precedent_coupe.to_csv('sortieprono.csv',header=True,index=False)

# Traitement des pronostiques

In [48]:
#Résultat prono coupe du monde 2010

In [49]:
prono2010=match_precedent_coupe[match_precedent_coupe['date']==2010]

In [50]:
phase_final2010=prono2010[prono2010['phase_final']==1]

In [51]:
phase_final2010.head(32)

,date,home_team,home_score,away_score,away_team,tournament,winner,phase_final,home_team_pourcentage_victoire,home_team_but_mis,...,away_team_but_enc,Rang_home_team,Rang_away_team,Host,PronovicPays1,Pronovicnul,PronovicPays2,pred,trouve,prono
48,2010,Uruguay,2,1,South Korea,1,2,1,55.56,1.9,...,0.7,16,47,0,1.70,3,4.40,2.0,1.0,0.70
49,2010,United States,1,2,Ghana,1,0,1,50.00,1.7,...,1.0,14,32,0,2.20,2.9,2.85,2.0,0.0,-1.00
50,2010,Argentina,3,1,Mexico,1,2,1,69.57,1.9,...,0.8,7,17,0,2.65,2.8,2.40,2.0,1.0,1.65
51,2010,Germany,4,1,England,1,2,1,66.67,2.4,...,0.8,6,8,0,2.65,2.8,2.40,2.0,1.0,1.65
52,2010,Brazil,3,0,Chile,1,2,1,83.33,2.3,...,0.9,1,18,0,1.38,3.8,6.00,2.0,1.0,0.38
53,2010,Netherlands,2,1,Slovakia,1,2,1,71.43,2.2,...,1.3,4,34,0,1.25,4.2,9.00,2.0,1.0,0.25
54,2010,Paraguay,5,3,Japan,1,2,1,37.50,1.1,...,0.9,31,45,0,2.00,3,3.20,2.0,1.0,1.00
55,2010,Spain,1,0,Portugal,1,2,1,84.62,2.7,...,0.5,2,3,0,1.83,3,3.70,0.0,0.0,-1.00
56,2010,Netherlands,2,1,Brazil,1,2,1,71.43,2.2,...,0.6,4,1,0,3.70,3.1,1.80,0.0,0.0,-1.00
57,2010,Uruguay,4,2,Ghana,1,2,1,55.56,1.9,...,1.0,16,32,0,1.75,3,4.09,2.0,1.0,0.75


# performance 2010

In [52]:
total2010=prono2010['prono'].sum()

In [53]:
totalphase_final2010=phase_final2010['prono'].sum()

In [54]:
#Affiche les performances des paris sportif sur la coupe du monde 2010

In [55]:
print(total2010)

-7.08


In [56]:
#Affiche les performances des paris sportif sur les phases finale de la coupe du monde 2010

In [57]:
print(totalphase_final2010)

6.11


In [58]:
#Permet de voir des meilleurs performances en ne pariant que sur les phases finales

In [59]:
test=prono2010[prono2010['winner']==1]

In [60]:
#Nombre de matchs bien prédit sur les 16 des phases finales de la CDM 2010

In [61]:
test['winner'].sum()

14

In [62]:
bonpari2010=((prono2010['trouve'].sum())/len(prono2010))*100
print('Le pourcentage de paris trouvés sur la coupe du monde 2010 est de : ',bonpari2010,' %')

Le pourcentage de paris trouvés sur la coupe du monde 2010 est de :  51.5625  %


# prono2014

In [63]:
prono2014=match_precedent_coupe[match_precedent_coupe['date']==2014]

In [64]:
phase_final2014=prono2014[prono2014['phase_final']==1]

In [65]:
totalphase_final2014=phase_final2014['prono'].sum()
print(totalphase_final2014)

8.28


In [66]:
total2014=prono2014['prono'].sum()

In [67]:
print(total2014)

13.91


In [68]:
bonpari2014=((prono2014['trouve'].sum())/len(prono2014))*100
print('Le pourcentage de paris trouvés sur la coupe du monde 2014 est de : ',bonpari2014,' %')

Le pourcentage de paris trouvés sur la coupe du monde 2014 est de :  64.0625  %


# prono2018

In [69]:
prono2018=match_precedent_coupe[match_precedent_coupe['date']==2018]

In [70]:
total2018=prono2018['prono'].sum()

In [71]:
phase_final2018=prono2018[prono2018['phase_final']==1]

In [72]:
totalphase_final2018=phase_final2018['prono'].sum()
print(totalphase_final2018)

1.62


In [73]:
print(total2018)

18.42


In [74]:
bonpari2018=((prono2018['trouve'].sum())/len(prono2018))*100
print('Le pourcentage de paris trouvés sur la coupe du monde 2018 est de : ',bonpari2018,' %')

Le pourcentage de paris trouvés sur la coupe du monde 2018 est de :  54.6875  %


In [75]:
#phase_final2010

In [76]:
#phase_final2014

In [77]:
#phase_final2018

# Coupe du Monde 2022

# Chargement du model

In [80]:
model=tf.keras.models.load_model('fifa2022.h5')

In [83]:
#Chargement des matchs de la coupe du monde

In [84]:
match_coupe2022=pd.read_excel("prono2022.xlsx")

In [85]:
#Nettoyage

In [86]:
poule=match_coupe2022

In [87]:
poule=poule.drop('winner',axis=1)
poule_Pays=match_coupe2022


In [88]:
poule=poule.drop('home_team',axis=1)

In [89]:
poule=poule.drop('away_team',axis=1)

In [90]:
poule['Rang_home_team']=(1.)/(poule['Rang_home_team'])

In [91]:
poule['Rang_away_team']=(1.)/(poule['Rang_away_team'])

In [92]:
X_pred=poule[['Rang_home_team','Rang_away_team','Host']].values

# Prédiction des matchs 2022

In [93]:
prediction=model.predict(X_pred)
i=0

for pred in prediction:
    print('Pour le match entre',poule_Pays['home_team'][i],'et',poule_Pays['away_team'][i])
    if (np.argmax(pred))==2 :

        print('Victoire pour',poule_Pays['home_team'][i],'\n')
        match_coupe2022['pred'][i]=2
        if match_coupe2022['winner'][i]==match_coupe2022['pred'][i] :
            match_coupe2022['trouve'][i]=1
        else :
            match_coupe2022['trouve'][i]=0
        
        if match_coupe2022['trouve'][i]==0. :
            match_coupe2022['prono'][i]=-1
        else :
            if match_coupe2022['pred'][i]==2. :
                match_coupe2022['prono'][i]=(match_coupe2022['PronovicPays1'][i])-1
            elif match_coupe2022['pred'][i]==1. :
                match_coupe2022['prono'][i]=(match_coupe2022['Pronovicnul'][i])-1
            elif match_coupe2022['pred'][i]==0. :
                match_coupe2022['prono'][i]=(match_coupe2022['PronovicPays2'][i])-1
                
                
                


    elif (np.argmax(pred))==0:
        print('Victoire pour',poule_Pays['away_team'][i],'\n')
        match_coupe2022['pred'][i]=0
        if match_coupe2022['winner'][i]==match_coupe2022['pred'][i] :
            match_coupe2022['trouve'][i]=1
        else :
            match_coupe2022['trouve'][i]=0
        if match_coupe2022['trouve'][i]==0. :
            match_coupe2022['prono'][i]=-1
        else :
            if match_coupe2022['pred'][i]==2. :
                match_coupe2022['prono'][i]=(match_coupe2022['PronovicPays1'][i])-1
            elif match_coupe2022['pred'][i]==1. :
                match_coupe2022['prono'][i]=(match_coupe2022['Pronovicnul'][i])-1
            elif match_coupe2022['pred'][i]==0. :
                match_coupe2022['prono'][i]=(match_coupe2022['PronovicPays2'][i])-1

    else :
        print('Match null\n')
        match_coupe2022['pred'][i]=1
        if match_coupe2022['winner'][i]==match_coupe2022['pred'][i] :
            match_coupe2022['trouve'][i]=1
        else :
            match_coupe2022['trouve'][i]=0
        if match_coupe2022['trouve'][i]==0. :
            match_coupe2022['prono'][i]=-1
        else :
            if match_coupe2022['pred'][i]==2. :
                match_coupe2022['prono'][i]=(match_coupe2022['PronovicPays1'][i])-1
            elif match_coupe2022['pred'][i]==1. :
                match_coupe2022['prono'][i]=(match_coupe2022['Pronovicnul'][i])-1
            elif match_coupe2022['pred'][i]==0. :
                match_coupe2022['prono'][i]=(match_coupe2022['PronovicPays2'][i])-1


    i=i+1

2/2 [==============================] - 0s 1ms/step
Pour le match entre Qatar et Ecuador
Victoire pour Qatar 

Pour le match entre England et Iran
Victoire pour England 

Pour le match entre Senegal et Netherlands
Victoire pour Netherlands 

Pour le match entre United States et Wales
Victoire pour United States 

Pour le match entre Argentina et Saudi Arabia
Victoire pour Argentina 

Pour le match entre Denmark et Tunisia
Victoire pour Denmark 

Pour le match entre Mexico et Poland
Victoire pour Mexico 

Pour le match entre France et Australia
Victoire pour France 

Pour le match entre Morocco et Croatia
Victoire pour Croatia 

Pour le match entre Germany et Japan
Victoire pour Germany 

Pour le match entre Spain et Costa Rica
Victoire pour Spain 

Pour le match entre Belgium et Canada
Victoire pour Belgium 

Pour le match entre Switzerland et Cameroon
Victoire pour Switzerland 

Pour le match entre Uruguay et South Korea
Victoire pour Uruguay 

Pour le match entre Portugal et Ghana
Vic

C:\Users\chris\AppData\Local\Temp\ipykernel_14032\98871231.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_coupe2022['pred'][i]=2
C:\Users\chris\AppData\Local\Temp\ipykernel_14032\98871231.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_coupe2022['trouve'][i]=0
C:\Users\chris\AppData\Local\Temp\ipykernel_14032\98871231.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_coupe2022['prono'][i]=-1
C:\Users\chri

In [94]:
match_coupe2022

,date,home_team,home_score,away_score,away_team,tournament,winner,phase_final,home_team_pourcentage_victoire,home_team_but_mis,...,away_team_but_enc,Rang_home_team,Rang_away_team,Host,PronovicPays1,Pronovicnul,PronovicPays2,pred,trouve,prono
0,2022,Qatar,0,2,Ecuador,1,0,0,50.00,1.4,...,1.0,150,44,1,3.40,3.15,2.30,2.0,0.0,-1.00
1,2022,England,6,2,Iran,1,2,0,68.00,2.3,...,0.4,5,20,0,1.35,4.50,10.50,2.0,1.0,0.35
2,2022,Senegal,0,2,Netherlands,1,0,0,62.96,1.6,...,1.0,18,8,0,6.25,3.65,1.60,0.0,1.0,0.60
3,2022,United States,1,1,Wales,1,1,0,68.75,2.0,...,1.2,16,19,0,2.40,3.10,3.10,2.0,0.0,-1.00
4,2022,Argentina,1,2,Saudi Arabia,1,0,0,68.18,1.9,...,0.6,3,51,0,1.10,10.00,26.00,2.0,0.0,-1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2022,England,1,2,France,1,0,1,NaN,NaN,...,NaN,5,4,0,2.90,3.15,2.75,2.0,0.0,-1.00
60,2022,Argentina,3,0,Croatia,1,2,1,NaN,NaN,...,NaN,3,12,0,2.00,3.10,4.35,2.0,1.0,1.00
61,2022,France,2,0,Morocco,1,2,1,NaN,NaN,...,NaN,4,22,0,1.59,4.10,6.00,2.0,1.0,0.59
62,2022,Croatia,2,1,Morocco,1,2,1,NaN,NaN,...,NaN,12,22,0,2.20,3.65,3.15,2.0,1.0,1.20


In [95]:
total2022=match_coupe2022['prono'].sum()

In [96]:
trouve2022=match_coupe2022['trouve'].sum()

In [97]:
#Affichage des resultats

# Résultat des paris pour 2022

In [98]:
print ("Pour la Coupe du monde 2022, nous avons trouvé "+ str(trouve2022)+" paris sur les : "+str(len(match_coupe2022)))

Pour la Coupe du monde 2022, nous avons trouvé 35.0 paris sur les : 64


In [99]:
print ("Soit "+str(trouve2022/(len(match_coupe2022))*100)+" %")

Soit 54.6875 %


In [100]:
print("Si 1€ parié à tous les matchs de la coupe le resultat final est : "+str(total2022))

Si 1€ parié à tous les matchs de la coupe le resultat final est : 2.759999999999998


# Optimisation

In [101]:
phase_final2022=match_coupe2022[match_coupe2022['phase_final']==1]

In [102]:
totalphase_final2022=phase_final2022['prono'].sum()
print(totalphase_final2022)

5.690000000000001


In [103]:
trouve_phase_final_2022=phase_final2022['trouve'].sum()
trouve_phase_final_2022

12.0

In [104]:
print ("Le pourcentage de matchs trouvés sur la phase finale est de : "+str(trouve_phase_final_2022/(len(phase_final2022))*100)+"%")

Le pourcentage de matchs trouvés sur la phase finale est de : 75.0%
